In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
cellrebel = pd.read_feather('Source Data/consolidate_file.feather')
# cellrebel = pd.read_csv('Source Data/cellrebel_20230227-20230310.csv', low_memory=False, parse_dates=[1])
grid = gpd.read_file('Source Data/POI_1.geojson')

In [3]:
cellrebel['mobile_operator'].replace('PT Telekomunikasi Indonesia', 'Telkomsel', inplace=True)
cellrebel['mobile_operator'].replace('3', 'Indosat Ooredoo', inplace=True)

In [4]:
cellrebel = cellrebel[cellrebel['mobile_operator'].str.contains('XL Axiata|Indosat Ooredoo|Telkomsel|Smartfren')== True ]
cellrebel = cellrebel[cellrebel['created_date'].astype(str).str.contains('2023-02-25|2023-02-26|2023-03-04|2023-03-05|2023-03-11|2023-03-12')==False]

In [5]:
cellrebel['created_date'] = pd.to_datetime(cellrebel['created_date'])

In [6]:
hour_filter = (cellrebel['created_date'].dt.hour == 7) | (cellrebel['created_date'].dt.hour == 8) | (cellrebel['created_date'].dt.hour == 9)| (cellrebel['created_date'].dt.hour == 18)

cellrebel = cellrebel[hour_filter]

In [7]:
cellrebel_cov = cellrebel[cellrebel['reference_signal_received_power'] < 0]
cellrebel_cap = cellrebel[cellrebel['dl_throughput'].astype(int) > 0]

In [8]:
cellrebel_cov['reference_signal_received_power'].sort_values()

1695677   -141.0
247639    -141.0
610109    -141.0
1170931   -141.0
1692768   -141.0
           ...  
939280     -44.0
938705     -44.0
726986     -44.0
585921     -44.0
1705427    -44.0
Name: reference_signal_received_power, Length: 214962, dtype: float64

In [9]:
cellrebel_gpd_cov = gpd.GeoDataFrame(cellrebel_cov, geometry=gpd.points_from_xy(cellrebel_cov.longitude, cellrebel_cov.latitude), crs=4326)
cellrebel_gpd_cap = gpd.GeoDataFrame(cellrebel_cap, geometry=gpd.points_from_xy(cellrebel_cap.longitude, cellrebel_cap.latitude), crs=4326)
grid = grid.to_crs(4326)
grid['geometry1'] = grid['geometry']

In [10]:
points_within_cov = gpd.sjoin(left_df= cellrebel_gpd_cov,right_df=grid, how='left', predicate='within')
points_within_cap = gpd.sjoin(left_df= cellrebel_gpd_cap,right_df=grid, how='left', predicate='within')

In [11]:
points_within_cap 

,Unnamed: 0,mobile_operator,created_date,latitude,longitude,province,kabupaten,reference_signal_received_power,dl_throughput,geometry,index_right,Polygon,Longitude,Latitude,geometry1
1471,1837,Smartfren,2023-03-15 09:20:00.011000+00:00,-6.308313,106.888830,Dki Jakarta,Kota Jakarta Timur,-107.0,7.707129,POINT (106.88883 -6.30831),NaN,NaN,NaN,NaN,None
1610,2030,Telkomsel,2023-03-15 08:44:00.065000+00:00,-6.330850,106.906270,Dki Jakarta,Kota Jakarta Timur,-104.0,11.235955,POINT (106.90627 -6.33085),NaN,NaN,NaN,NaN,None
1638,2078,Telkomsel,2023-03-15 08:50:00.255000+00:00,-6.297060,106.871753,Dki Jakarta,Kota Jakarta Timur,-94.0,6.962576,POINT (106.87175 -6.29706),NaN,NaN,NaN,NaN,None
1671,2124,Telkomsel,2023-03-15 07:28:00.084000+00:00,-6.243058,106.876125,Dki Jakarta,Kota Jakarta Timur,-93.0,1.828571,POINT (106.87613 -6.24306),NaN,NaN,NaN,NaN,None
1683,2148,Telkomsel,2023-03-15 08:44:00.032000+00:00,-6.173047,106.881544,Dki Jakarta,Kota Jakarta Timur,-99.0,24.539877,POINT (106.88154 -6.17305),NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818408,43710,Smartfren,2023-02-21 18:46:00.105000+00:00,-6.335222,106.879906,Dki Jakarta,Kota Jakarta Timur,-109.0,5.015674,POINT (106.87991 -6.33522),NaN,NaN,NaN,NaN,None
1818412,43714,Smartfren,2023-02-21 08:20:00.107000+00:00,-6.309502,106.898770,Dki Jakarta,Kota Jakarta Timur,-90.0,19.230769,POINT (106.89877 -6.30950),NaN,NaN,NaN,NaN,None
1818420,43727,Smartfren,2023-02-21 09:07:40.042000+00:00,-6.241613,106.897872,Dki Jakarta,Kota Jakarta Timur,-88.0,5.979073,POINT (106.89787 -6.24161),NaN,NaN,NaN,NaN,None
1818422,43730,Smartfren,2023-02-21 09:44:00.115000+00:00,-6.213014,106.899765,Dki Jakarta,Kota Jakarta Timur,-87.0,16.427105,POINT (106.89977 -6.21301),NaN,NaN,NaN,NaN,None


In [12]:
pivot_cov = pd.pivot_table(points_within_cov, index= ['Polygon', 'index_right'], columns=['mobile_operator'], values=['reference_signal_received_power'], aggfunc = np.mean)
pivot_cap = pd.pivot_table(points_within_cap, index= ['Polygon', 'index_right'], columns=['mobile_operator'], values=['dl_throughput'], aggfunc = lambda x: x.quantile(0.1))

In [13]:
pivot_cov.fillna(-150, inplace=True)
pivot_cap.fillna(0, inplace=True)

In [14]:
pivot_cov.reset_index(inplace=True, col_level=1, allow_duplicates=False)
pivot_cap.reset_index(inplace=True, col_level=1, allow_duplicates=False)

In [15]:
pivot_cov = pivot_cov.droplevel(level=0, axis=1)
pivot_cap = pivot_cap.droplevel(level=0, axis=1)

In [16]:
pivot_cov = pivot_cov.merge(points_within_cov[['index_right', 'geometry1']], how='left', on='index_right')
pivot_cap = pivot_cap.merge(points_within_cap[['index_right', 'geometry1']], how='left', on='index_right')

In [17]:
pivot_cov.drop_duplicates(ignore_index=True, inplace=True)
pivot_cap.drop_duplicates(ignore_index=True, inplace=True)

In [18]:
pivot_cov.to_csv('result/p10_poi_coverage_result.csv')
pivot_cap.to_csv('result/p10_poi_capacity_result.csv')
cellrebel.to_csv('processed_data.csv')